In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import data
import mne
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import scipy
import pingouin as pg

sns.set(font_scale=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data

In [16]:
path = 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue-questionnaire/Raw Data/'
xdf_file_1 = path + "S10_2022-05-12.xdf"
#xdf_file_2 = path + "S11_2022-05-12.xdf"
xdf_files = [xdf_file_1]


In [17]:
xdf_files

['C:/Users/bribeiroriccioppo/Desktop/mental-fatigue-questionnaire/Raw Data/S10_2022-05-12.xdf']

## Tone Task Data

In [22]:
def tone_preprocess(subject):
    
    tone_data = subjects[str(subject)]['tone_events']
    keyboard_data = subjects[str(subject)]['keyboard']
    
    tone_timestamps = tone_data['LSL timestamp'].to_numpy()
    ts_exp_start = tone_timestamps[0]
    ts_exp_end = tone_timestamps[-1]
    
    #blocks = {2: "Practice", 7: "Single1", 8: "Single2", 9: "Dual1", 10: "Dual2", 11: "Multi1", 12: "Multi2"}
    blocks = {7: "Single1", 8: "Single2", 9: "Dual1", 10: "Dual2", 11: "Multi1", 12: "Multi2"}
    tone_data = tone_data.replace({'block': blocks})
    #print(list(blocks.values()))
    tone_data = tone_data[tone_data['block'].isin(list(blocks.values()))]
    
    #keep only SPACE pressed events
    keyboard_data = keyboard_data[keyboard_data.Event.str.contains("SPACE pressed")] 
    #remove events that take place before first tone or more than 2 seconds after last tone
    keyboard_data = keyboard_data.loc[(keyboard_data['LSL timestamp'] >= ts_exp_start) & (keyboard_data['LSL timestamp'] <= ts_exp_end+2)]
    
    # assign space pressed events within 1.5 seconds following a tone, to that tone
    reaction_window = 1.5 #seconds
    space_ts = np.empty(tone_data['LSL timestamp'].shape)
    space_ts[:] = np.nan
    for idx, tone_ts in enumerate(tone_data['LSL timestamp']):
        for press_ts in keyboard_data['LSL timestamp']:
            if (press_ts>= tone_ts) & (press_ts <= tone_ts+reaction_window):
                space_ts[idx] = press_ts
                
    tone_data['space_ts'] = space_ts
    tone_data['reaction_time'] = tone_data['space_ts'] - tone_data['LSL timestamp']
    
    target_tone_data = tone_data.loc[tone_data['tone_id'] == 0]
    target_tone_data['miss'] = target_tone_data['reaction_time'].isna() #miss: space not pressed after target tone
    standard_tone_data = tone_data.loc[tone_data['tone_id'] == 1]
    standard_tone_data['FP'] = ~standard_tone_data['reaction_time'].isna() #false positives: space pressed on standard tone
    
    subjects[str(subject)]['tone_events'] = tone_data
    subjects[str(subject)]['keyboard'] = keyboard_data
    
    return tone_data, keyboard_data, target_tone_data, standard_tone_data

In [23]:
all_subjs_rt = []
all_subjs_miss = []
all_subjs_fp = []
for subject in subjects_list:
    print(subject)
    
    tone_data, _, target_tone_data, standard_tone_data = tone_preprocess(subject)
    
    all_subjs_rt.append(tone_data)
    all_subjs_miss.append(target_tone_data)
    all_subjs_fp.append(standard_tone_data)
    
    plot_rt = sns.barplot(x="block", y="reaction_time", data=tone_data)
    plot_rt.set_title(str(subject) + " Reaction Time", fontsize=15)
    plot_rt.set_xlabel('Block', fontsize=12)
    plot_rt.set_ylabel('Reaction Time (s)', fontsize=12)
    plt.show()
    
df_rt_all = pd.concat(all_subjs_rt).reset_index(drop=True)
df_miss_all = pd.concat(all_subjs_miss).reset_index(drop=True)
df_fp_all = pd.concat(all_subjs_fp).reset_index(drop=True)  

S10


KeyError: 'tone_events'

## Math Task Data